In [57]:
import os
import sys
from itertools import combinations
from node import Node

In [58]:
def frequent(current_node, items):

    items = set(items)

    while current_node.children:
        if current_node.item:
            items.add(current_node.item)
        current_node = current_node.children[0]

    patterns = []

    for length in range(2, len(items) + 1):
        patterns += combinations(items, length)

    return patterns



In [59]:
def init_count_map(transactions):
    count_map = {}

    for transaction in transactions:
        for item in transaction:
            count_map.setdefault(item, 1)
            count_map[item] += 1
    return count_map

In [60]:
def init_conditional_pattern(node_items, items):
    conditional_pattern = {_id: {} for _id in items}
    for _id, item_nodes in node_items:
        for item_node in item_nodes:
            conditional_pattern[_id].update(item_node.pattern())
    
    return conditional_pattern

In [61]:
def backtrack(node_links, items, conditional_items = None):
    if not conditional_items:
        conditional_items = []

    node_items = node_links.items()

    conditional_pattern = init_conditional_pattern(node_items, items)
    for _id, item_nodes in node_items:
        for item_node in item_nodes:
            conditional_pattern[_id].update(item_node.pattern())

    # generate fpt
    patterns = set()

    for _id, cond_pattern in conditional_pattern.items():
        tree = Node()
        tree_nodes = {_id: [] for _id in items}

        count_map = init_count_map(cond_pattern)
        for conditional_pattern_base in cond_pattern:
            tree.append(sorted(
                list(conditional_pattern_base),
                key=lambda item: count_map[item],
                reverse=True
            ), tree_nodes)

        if tree.is_single_path():
            for pattern in frequent(tree, conditional_items + [_id]):
                patterns.add(pattern)
        else:
            patterns = patterns.union(backtrack(tree_nodes, items, conditional_items + [_id]))

    return patterns

In [62]:
def association_to_string(transactions,itemset,subset,min_sup, min_conf):
    try:
        seen = association_to_string._seen
    except AttributeError:
        seen = []
        association_to_string._seen = seen

    keys = (tuple(sorted(itemset)), tuple(sorted(subset)))
    if keys in seen:
        return
    else:
        seen.append(keys)

    count_l = 0
    count_s = 0

    for transaction in transactions:
        if all(item in transaction for item in itemset):
            count_l += 1

        if all(item in transaction for item in subset):
            count_s += 1

    support = count_l / len(transactions)
    confidence = count_l / count_s

    if support >= min_sup and confidence > min_conf:
        print(f"Association Rule: l: {subset} s :{itemset.difference(subset)} --> support:{round(support, 3)}, confidence:{round(confidence, 3)}")


In [55]:
def init_root_transaction(root, transactions_set, count_map, node_links):
    for transaction in transactions_set:
        root.append(sorted(transaction, key=lambda item: count_map[item], reverse=True), node_links)

def read_input(input_file):
    transactions_set = []
    with open(input_file, "r") as file:
        for line in file:
            transactions_set.append([item for item in line.strip().split(",") if item])
    
    return transactions_set

def constuct_union_subset(pattern):
    subsets = set()
    for length in range(1, len(pattern)):
            subsets = subsets.union(set(combinations(pattern, length)))
    return subsets

In [56]:
min_sup = 0.4
min_conf = 0.6
input_file = "data.csv" 


transactions_set = read_input(input_file)


count_map = init_count_map(transactions_set)
node_links = {_id: [] for _id in count_map.keys()}

root = Node()
init_root_transaction(root, transactions_set, count_map, node_links)

frequent = backtrack(node_links, items=list(count_map.keys()))
for pattern in frequent:
    subsets = constuct_union_subset(pattern)

    pattern = set(pattern)
    for subset in subsets:    
        subset = set(subset)
        association_to_string(transactions_set, pattern, subset, min_sup, min_conf)

TypeError: 'set' object is not callable